In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
approved_list = ["adhd_2018_features_tfidf_256.csv",
                "bpd_2019_features_tfidf_256.csv",
                "schizophrenia_2019_features_tfidf_256.csv",
                "bipolarreddit_2018_features_tfidf_256.csv",
                "anxiety_2019_features_tfidf_256.csv",
                "depression_2019_features_tfidf_256.csv",
                "EDAnonymous_2019_features_tfidf_256.csv"]

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install --upgrade gensim

In [ ]:
import spacy
sp = spacy.load("en_core_web_sm")
import os 
import csv
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from IPython.display import display, HTML
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import gensim
from scipy import spatial
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import time
from sklearn.decomposition import PCA
import re
from statistics import mean
# to make our plot outputs appear and be stored within the notebook:
%matplotlib inline 

In [ ]:
posts_dict = {}
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename in approved_list:
            curr_df = pd.read_csv('/kaggle/input/mental-health-support-feature-analysis/' + filename)
#             print(curr_df.columns)
#             print(curr_df.post)
            print(len(list(curr_df.post)))
            disorder = filename.split("_")[0]
            print("DISORDER: " + disorder)
            if disorder == "bipolarreddit":
                disorder = "bipolar"
            elif disorder == "EDAnonymous":
                disorder = "eating_disorder"
            posts_dict[disorder] = list(curr_df.post)
    
print("FINISHED LOOPING THROUGH FILES")
for k in posts_dict.keys():
    print(f"Key: {k}\nLength: {len(posts_dict[k])}")

In [ ]:
df = pd.DataFrame(columns=['disorder', 'post'])
count = 0
for k in posts_dict.keys():
    for post in posts_dict[k]:
        df.loc[count] = [k, post]
        count += 1
print(df.shape)

In [ ]:
print(df.head)
shortened_set = df.sample(frac=0.3)
shortened_set = shortened_set.reset_index().drop('index', axis=1)
print(shortened_set.head)

In [ ]:
shortened_set['post'] = [sp(shortened_set['post'].iloc[i]) for i in range(shortened_set.shape[0])]
display(shortened_set.head)

In [ ]:
shortened_set['tokens'] = [[word.text for word in shortened_set['post'].iloc[i]] for i in range(shortened_set.shape[0])]

In [ ]:
shortened_set['lemmas'] = [[word.lemma_ for word in shortened_set['post'].iloc[i]] for i in range(shortened_set .shape[0])]
display(shortened_set)

In [ ]:
for i in range(shortened_set.shape[0]):
    shortened_set['lemmas'].iloc[i] = [t for t in shortened_set['lemmas'].iloc[i] if t not in '''!()-[]{};:\'"\,<>./?@#$%^&*_~\|–—\“’`''']# and t.isalpha() ]
shortened_set

In [ ]:
stop_words = set(stopwords.words('english'))
shortened_set["lemmas_no_stop"] = ''
for i in range(shortened_set.shape[0]):
    shortened_set['lemmas_no_stop'].iloc[i] = [t for t in shortened_set['lemmas'].iloc[i] if t.lower() not in stop_words and t.lower() not in ["i", "is", "be", "am", "are", "the", "reddit", "i'm", "i’m"]]
shortened_set

In [ ]:
def print_vocab(model, top_n = None):
    count = 0
    if top_n is not None:
        for index, word in enumerate(model.wv.index_to_key):
            count+= 1
            if count < top_n:
                print(f"WORD #{index}/{len(model.wv.index_to_key)} IS: {word}")

In [ ]:
cbow_models = {}
# 100 dimensions
cbow_w3_f1_100 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0)
cbow_models["cbow_w3_f1_100"] = cbow_w3_f1_100
cbow_w5_f1_100 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=100, window = 5, sg = 0)
cbow_models["cbow_w5_f1_100"] = cbow_w5_f1_100
cbow_w10_f1_100 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0)
cbow_models["cbow_w10_f1_100"] = cbow_w10_f1_100
cbow_w3_f1_100_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0)
cbow_models["cbow_w3_f1_100_mc0"] = cbow_w3_f1_100_mc0
cbow_w5_f1_100_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=100, window = 5, sg = 0)
cbow_models["cbow_w5_f1_100_mc0"] = cbow_w5_f1_100_mc0
cbow_w10_f1_100_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0)
cbow_models["cbow_w10_f1_100_mc0"] = cbow_w10_f1_100_mc0
cbow_w10_f1_100_neg2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75)
cbow_models["cbow_w10_f1_100_neg2"] = cbow_w10_f1_100_neg2
cbow_w10_f1_100_neg5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75)
cbow_models["cbow_w10_f1_100_neg5"] = cbow_w10_f1_100_neg5
cbow_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5)
cbow_models["cbow_w10_f1_100_ns_half_neg2"] = cbow_w10_f1_100_ns_half_neg2
cbow_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5)
cbow_models["cbow_w10_f1_100_ns_half_neg5"] = cbow_w10_f1_100_ns_half_neg5
cbow_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75)
cbow_models["cbow_w10_f1_100_neg2_ws2"] = cbow_w10_f1_100_neg2_ws2
cbow_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=100, window = 5, sg = 0, negative=5, ns_exponent=0.75)
cbow_models["cbow_w10_f1_100_neg5_ws5"] = cbow_w10_f1_100_neg5_ws5
print("FINISHED TRAINING 100 DIMENSION---WHITE")

# 300 dimensions
cbow_w3_f1_300 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0)
cbow_models["cbow_w3_f1_300"] = cbow_w3_f1_300
cbow_w5_f1_300 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=300, window = 5, sg = 0)
cbow_models["cbow_w5_f1_300"] = cbow_w5_f1_300
cbow_w10_f1_300 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0)
cbow_models["cbow_w10_f1_300"] = cbow_w10_f1_300
cbow_w3_f1_300_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0)
cbow_models["cbow_w3_f1_300_mc0"] = cbow_w3_f1_300_mc0
cbow_w5_f1_300_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=300, window = 5, sg = 0)
cbow_models["cbow_w5_f1_300_mc0"] = cbow_w5_f1_300_mc0
cbow_w10_f1_300_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0)
cbow_models["cbow_w10_f1_300_mc0"] = cbow_w10_f1_300_mc0
cbow_w10_f1_300_neg2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75)
cbow_models["cbow_w10_f1_300_neg2"] = cbow_w10_f1_300_neg2
cbow_w10_f1_300_neg5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75)
cbow_models["cbow_w10_f1_300_neg5"] = cbow_w10_f1_300_neg5
cbow_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5)
cbow_models["cbow_w10_f1_300_ns_half_neg2"] = cbow_w10_f1_300_ns_half_neg2
cbow_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5)
cbow_models["cbow_w10_f1_300_ns_half_neg5"] = cbow_w10_f1_300_ns_half_neg5
cbow_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75)
cbow_models["cbow_w10_f1_300_neg2_ws2"] = cbow_w10_f1_300_neg2_ws2
cbow_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=300, window = 5, sg = 0, negative=5, ns_exponent=0.75)
cbow_models["cbow_w10_f1_300_neg5_ws5"] = cbow_w10_f1_300_neg5_ws5
print("FINISHED TRAINING 300 DIMENSION---WHITE")

# 500 dimensions
cbow_w3_f1_500 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0)
cbow_models["cbow_w3_f1_500"] = cbow_w3_f1_500
cbow_w5_f1_500 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=500, window = 5, sg = 0)
cbow_models["cbow_w5_f1_500"] = cbow_w5_f1_500
cbow_w10_f1_500 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0)
cbow_models["cbow_w10_f1_500"] = cbow_w10_f1_500
cbow_w3_f1_500_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0)
cbow_models["cbow_w3_f1_500_mc0"] = cbow_w3_f1_500_mc0
cbow_w5_f1_500_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=500, window = 5, sg = 0)
cbow_models["cbow_w5_f1_500_mc0"] = cbow_w5_f1_500_mc0
cbow_w10_f1_500_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0)
cbow_models["cbow_w10_f1_500_mc0"] = cbow_w10_f1_500_mc0
cbow_w10_f1_500_neg2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75)
cbow_models["cbow_w10_f1_500_neg2"] = cbow_w10_f1_500_neg2
cbow_w10_f1_500_neg5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75)
cbow_models["cbow_w10_f1_500_neg5"] = cbow_w10_f1_500_neg5
cbow_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5)
cbow_models["cbow_w10_f1_500_ns_half_neg2"] = cbow_w10_f1_500_ns_half_neg2
cbow_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5)
cbow_models["cbow_w10_f1_500_ns_half_neg5"] = cbow_w10_f1_500_ns_half_neg5
cbow_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75)
cbow_models["cbow_w10_f1_500_neg2_ws2"] = cbow_w10_f1_500_neg2_ws2
cbow_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=500, window = 5, sg = 0, negative=5, ns_exponent=0.75)
cbow_models["cbow_w10_f1_500_neg5_ws5"] = cbow_w10_f1_500_neg5_ws5
print("FINISHED TRAINING 500 DIMENSION---WHITE")

# 700 dimensions
cbow_w3_f1_700 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0)
cbow_models["cbow_w3_f1_700"] = cbow_w3_f1_700
cbow_w5_f1_700 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=700, window = 5, sg = 0)
cbow_models["cbow_w5_f1_700"] = cbow_w5_f1_700
cbow_w10_f1_700 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0)
cbow_models["cbow_w10_f1_700"] = cbow_w10_f1_700
cbow_w3_f1_700_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0)
cbow_models["cbow_w3_f1_700_mc0"] = cbow_w3_f1_700_mc0
cbow_w5_f1_700_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=700, window = 5, sg = 0)
cbow_models["cbow_w5_f1_700_mc0"] = cbow_w5_f1_700_mc0
cbow_w10_f1_700_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0)
cbow_models["cbow_w10_f1_700_mc0"] = cbow_w10_f1_700_mc0
cbow_w10_f1_700_neg2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75)
cbow_models["cbow_w10_f1_700_neg2"] = cbow_w10_f1_700_neg2
cbow_w10_f1_700_neg5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75)
cbow_models["cbow_w10_f1_700_neg5"] = cbow_w10_f1_700_neg5
cbow_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5)
cbow_models["cbow_w10_f1_700_ns_half_neg2"] = cbow_w10_f1_700_ns_half_neg2
cbow_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5)
cbow_models["cbow_w10_f1_700_ns_half_neg5"] = cbow_w10_f1_700_ns_half_neg5
cbow_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75)
cbow_models["cbow_w10_f1_700_neg2_ws2"] = cbow_w10_f1_700_neg2_ws2
cbow_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=700, window = 5, sg = 0, negative=5, ns_exponent=0.75)
cbow_models["cbow_w10_f1_700_neg5_ws5"] = cbow_w10_f1_700_neg5_ws5
print("FINISHED TRAINING 700 DIMENSION---WHITE")

# 1000 dimensions
cbow_w3_f1_1000 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0)
cbow_models["cbow_w3_f1_1000"] = cbow_w3_f1_1000
cbow_w5_f1_1000 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=1000, window = 5, sg = 0)
cbow_models["cbow_w5_f1_1000"] = cbow_w5_f1_1000
cbow_w10_f1_1000 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0)
cbow_models["cbow_w10_f1_1000"] = cbow_w10_f1_1000
cbow_w3_f1_1000_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0)
cbow_models["cbow_w3_f1_1000_mc0"] = cbow_w3_f1_1000_mc0
cbow_w5_f1_1000_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=1000, window = 5, sg = 0)
cbow_models["cbow_w5_f1_1000_mc0"] = cbow_w5_f1_1000_mc0
cbow_w10_f1_1000_mc0 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0)
cbow_models["cbow_w10_f1_1000_mc0"] = cbow_w10_f1_1000_mc0
cbow_w10_f1_1000_neg2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75)
cbow_models["cbow_w10_f1_1000_neg2"] = cbow_w10_f1_1000_neg2
cbow_w10_f1_1000_neg5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75)
cbow_models["cbow_w10_f1_1000_neg5"] = cbow_w10_f1_1000_neg5
cbow_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5)
cbow_models["cbow_w10_f1_1000_ns_half_neg2"] = cbow_w10_f1_1000_ns_half_neg2
cbow_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5)
cbow_models["cbow_w10_f1_1000_ns_half_neg5"] = cbow_w10_f1_1000_ns_half_neg5
cbow_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75)
cbow_models["cbow_w10_f1_1000_neg2_ws2"] = cbow_w10_f1_1000_neg2_ws2
cbow_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(shortened_set['lemmas_no_stop'], min_count=1, vector_size=1000, window = 5, sg = 0, negative=5, ns_exponent=0.75)
cbow_models["cbow_w10_f1_1000_neg5_ws5"] = cbow_w10_f1_1000_neg5_ws5
print("FINISHED TRAINING 1000 DIMENSION---WHITE")
print("FINISHED TRAINING ALL WORD2VEC MODELS")

In [ ]:
synonyms = dict({"smart": "intelligent", "happy": "cheerful", "afraid": "anxious", "large": "big", "insane": "crazy", "weird" : "strange", "bad" : "terrible", })

In [ ]:
models_synonymity_average = dict()
cbow_names = list(cbow_models.keys())
cbow_vals = list(cbow_models.values())
for i in range(len(cbow_names)):
    print("Model", cbow_names[i])
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        print("\tSimilarity between", s1, "and", synonyms[s1],  "in", cbow_names[i], cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    synonymities.append(cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    print(average_synonimity)
    models_synonymity_average[cbow_names[i]] = average_synonimity
models_synonymity_average

In [ ]:
model_chosen_name = max(models_synonymity_average, key=models_synonymity_average.get)
print(model_chosen_name)

In [ ]:
# Winning model
print_vocab(cbow_models["cbow_w10_f1_500_ns_half_neg2"], top_n = 20)
# the following will find words used in similar context to listed words, but does not
# necessarily qualify or define the words in the list
for word in ['black', 'white', 'misdiagnosis', 'misdiagnosed', 'ethnicity', 'ethnic', 'indian', 'asian', 'caucasian', 'african', 'mexican', 'chinese', 'muslim', 'spanish', 'english', 'native', 'racist', 'bigot', 'bigoted', 'discrimination', 'racism', 'stereotype', 'stereotypical', 'family', 'culture', 'cultural', 'aggression', 'hostility', 'hostile', 'involuntary', 'forced', 'aggressive', 'hostile', 'violent', 'restrained', 'stereotyped', 'bias', 'biased', 'nhs', 'religious', 'condescending', 'disbelief', 'immigrant', 'foreign', 'foreigners', 'indigenous', 'ghetto', 'slang', 'aave']:
    try:
        print(f"\nWORD: {word.upper()}")
        print(cbow_w10_f1_500_ns_half_neg2, cbow_w10_f1_500_ns_half_neg2.wv.similar_by_word(word, 30))
        print("\n")
    except:
        print(f"WORD: {word}")
        print(f"no instance of {word} found\n\n")
        
# TODO: print concordances for each word in the list

# # # # SECTION II: Separate by Disorder Category + Cosine Similarity

In [ ]:
df_depression = df[df['disorder'] == 'depression'].sample(frac=0.19689)
df_anxiety = df[df['disorder'] == 'anxiety'].sample(frac=0.5)
display(df_depression)
display(df_anxiety)

In [ ]:
df_depression['disorder'] = [sp(df_depression['post'].iloc[i]) for i in range(df_depression.shape[0])]

df_anxiety['disorder'] = [sp(df_anxiety['post'].iloc[i]) for i in range(df_anxiety.shape[0])]

df_depression['tokens'] = [[word.text for word in sp(df_depression['post'].iloc[i])] for i in range(df_depression.shape[0])]
display(df_depression)
df_anxiety['tokens'] = [[word.text for word in sp(df_anxiety['post'].iloc[i])] for i in range(df_anxiety.shape[0])]

df_depression['lemmas'] = [[word.lemma_ for word in sp(df_depression['post'].iloc[i])] for i in range(df_depression .shape[0])]
display(df_depression)
df_anxiety['lemmas'] = [[word.lemma_ for word in sp(df_anxiety['post'].iloc[i])] for i in range(df_anxiety .shape[0])]

for i in range(df_depression.shape[0]):
    df_depression['lemmas'].iloc[i] = [t for t in df_depression['lemmas'].iloc[i] if t not in '''!()-[]{};:\'"\,<>./?@#$%^&*_~\|–—\“’`''']
display(df_depression)
for i in range(df_anxiety.shape[0]):
    df_anxiety['lemmas'].iloc[i] = [t for t in df_anxiety['lemmas'].iloc[i] if t not in '''!()-[]{};:\'"\,<>./?@#$%^&*_~\|–—\“’`''']
    
stop_words = set(stopwords.words('english'))
df_depression["lemmas_no_stop"] = ''
for i in range(df_depression.shape[0]):
    df_depression['lemmas_no_stop'].iloc[i] = [t for t in df_depression['lemmas'].iloc[i] if t.lower() not in stop_words and t.lower() not in ["i", "is", "be", "am", "are", "the", "reddit", "i'm", "i’m"]]
display(df_depression)
df_anxiety["lemmas_no_stop"] = ''
for i in range(df_anxiety.shape[0]):
    df_anxiety['lemmas_no_stop'].iloc[i] = [t for t in df_anxiety['lemmas'].iloc[i] if t.lower() not in stop_words and t.lower() not in ["i", "is", "be", "am", "are", "the", "reddit", "i'm", "i’m"]]

display(df_depression)
display(df_anxiety)

In [ ]:
model_dep = gensim.models.Word2Vec(df_depression['lemmas_no_stop'], min_count=1, vector_size=300, window = 5, sg = 1)
model_anx = gensim.models.Word2Vec(df_anxiety['lemmas_no_stop'], min_count=1, vector_size=300, window = 5, sg = 1)

In [ ]:
print_vocab(model_dep, 10)
print_vocab(model_anx, 10)

In [ ]:
# measure quality of embeddings:
    # standard way to evaluate word2vec is to use a fixed list like https://fh295.github.io/simlex.html
    # calculate cosine similarities between pairs of words as calculated by each model, figure out best model from that
# next step, split corpus by condition/other defining factor:
    # 
# HOW TO CHOOSE WORDS:
    # focus on words that we know are of particular interest:
        # culture, family
        # aggressiveness
        # could use https://wordnet.princeton.edu/ to
         # find synonyms and related words for concepts
         # like culture/family/aggressiveness
        # check synonyms in word embeddings for words like aggressive
         # to see whether they're different for different races
         # such as hostile (pos) vs assertive (neg)
        # http://sag.art.uniroma2.it/demo-software/distributional-polarity-lexicon/
# how to write code in order to produce results I can thoroughly analyse

# strong dissertation advice
    # write a dissertation which is understandable by
     # people who know nothing about the computational
     # side, but also explain the technological methodology
     # well and ensure it's sound
    # RICH literature review, talk about what's not been done
     # one aspect = using non-chronological semantic shift (comp ling innovation)
     # ethnicity + mental health using semantic shift = new angle
    # one thing which is overlooked: leave enough time to reflect on results
     # need multiple weeks to do so